In [1]:
import pandas as pd
import quandl

In [23]:
import math
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection #cross_validation
from sklearn import svm
from sklearn.linear_model import LinearRegression

In [5]:
df = quandl.get('WIKI/GOOGL')

In [6]:
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [9]:
df = df[['Adj. Open','Adj. High', 'Adj. Low', 'Adj. Close','Adj. Volume']]

In [10]:
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume
Date                                                                
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0


In [16]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

C:\Users\Big Chungus\AppData\Local\Temp\ipykernel_2484\3793859311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
C:\Users\Big Chungus\AppData\Local\Temp\ipykernel_2484\3793859311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0


In [17]:
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume  \
Date                                                                   
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0   
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0   
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0   
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0   
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0   

              HL_PCT  PCT_change  
Date                              
2004-08-19  3.712563    0.324968  
2004-08-20  0.710922    7.227007  
2004-08-23  3.729433   -1.227880  
2004-08-24  6.417469   -5.726357  
2004-08-25  1.886792    1.183658  


In [20]:
df1 = df[['Adj. Close', 'HL_PCT','PCT_change','Adj. Volume']]

In [22]:
print(df1.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  3.712563    0.324968   44659000.0
2004-08-20   54.322689  0.710922    7.227007   22834300.0
2004-08-23   54.869377  3.729433   -1.227880   18256100.0
2004-08-24   52.597363  6.417469   -5.726357   15247300.0
2004-08-25   53.164113  1.886792    1.183658    9188600.0


In [24]:
forecast_col = 'Adj. Close'

In [25]:
df.fillna(-99999, inplace = True) #don't remove data with NAN values, replace with -99999

C:\Users\Big Chungus\anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [27]:
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume  \
Date                                                                   
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0   
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0   
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0   
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0   
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0   

              HL_PCT  PCT_change  
Date                              
2004-08-19  3.712563    0.324968  
2004-08-20  0.710922    7.227007  
2004-08-23  3.729433   -1.227880  
2004-08-24  6.417469   -5.726357  
2004-08-25  1.886792    1.183658  


In [34]:
forecast_out = int(math.ceil(0.01*len(df)))

In [35]:
print(forecast_out)

35


In [40]:
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)

C:\Users\Big Chungus\AppData\Local\Temp\ipykernel_2484\3984670937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df[forecast_col].shift(-forecast_out)
C:\Users\Big Chungus\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [41]:
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume  \
Date                                                                   
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0   
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0   
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0   
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0   
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0   

              HL_PCT  PCT_change      label  
Date                                         
2004-08-19  3.712563    0.324968  69.078238  
2004-08-20  0.710922    7.227007  67.839414  
2004-08-23  3.729433   -1.227880  68.912727  
2004-08-24  6.417469   -5.726357  70.668146  
2004-08-25  1.886792    1.183658  71.219849  


In [42]:
print(df.tail())

            Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume    HL_PCT  \
Date                                                                            
2018-01-30    1177.72    1187.93   1174.51     1177.37    1792602.0  0.896914   
2018-01-31    1183.81    1186.32   1172.10     1182.22    1643877.0  0.346805   
2018-02-01    1175.99    1187.45   1169.36     1181.59    2774967.0  0.495942   
2018-02-02    1127.42    1131.30   1111.17     1119.20    5798880.0  1.081129   
2018-02-05    1100.61    1114.99   1056.74     1068.76    3742469.0  4.325574   

            PCT_change    label  
Date                             
2018-01-30   -0.029718  1094.00  
2018-01-31   -0.134312  1053.15  
2018-02-01    0.476195  1026.55  
2018-02-02   -0.729098  1054.09  
2018-02-05   -2.893850  1006.94  


In [79]:
X = np.array(df.drop(['label'],1))
y = np.array(df['label'])

C:\Users\Big Chungus\AppData\Local\Temp\ipykernel_2484\3153512655.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(df.drop(['label'],1))


In [80]:
X = preprocessing.scale(X)

In [81]:
y = np.array(df['label'])

In [82]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2)

In [83]:
clf = svm.SVR(kernel = 'linear') #LinearRegression()
clf.fit(X_train, y_train)

SVR(kernel='linear')

In [84]:
accuracy = clf.score(X_test,y_test)

In [85]:
print(accuracy)

0.9789986066687499
